<IMG SRC="https://github.com/jacquesroy/byte-size-data-science/raw/master/images/Banner.png" ALT="BSDS Banner" WIDTH=1195 HEIGHT=200>

<table align="left">
    <tr><td>
<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a></td><td>This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.</td>
    </tr>
    <tr><td>Jacques Roy, Byte Size Data Science</td><td> </td></tr>
    </table>

# Data Understanding and Preparation (part 2)
To understand data, we need to explore it.

This adds to the following videos:
- <a href="https://youtu.be/xSDP6u_Xqhc">017-Spark Data Exploration</a>
- <a href="https://youtu.be/AeeHapnLhyE">018-Python Pandas Data Exploration</a>
- <a href="https://youtu.be/qw4FtewQFZE">032-JDBC Data Exploration</a>
- <a href="https://youtu.be/qw4FtewQFZE">060-Data Understanding and Preparation</a>

## 060-Data Understanding and Preparation
Execute the next cell if you want to see the `Byte Size Data Science` youtube channel video

In [ ]:
from IPython.display import IFrame

IFrame(src="https://www.youtube.com/embed/wJQ-5Cm1H0E?rel=0&amp;controls=0&amp;showinfo=0", width=560, height=315)


## Import the appropriate libraries and set up needed connections
There is another library to connect to db2. See: https://pythonhosted.org/ibmdbpy/

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import ibm_db
import ibm_db_dbi
import math

from ftplib import FTP
import requests, zipfile, io

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
credentials = {
    'username': 'bluadmin',
    'password': """PASSWORD""",
    'sg_service_url': 'https://sgmanager.ng.bluemix.net',
    'database': 'BLUDB',
    'host': 'dashdb-enterprise-. . . .bluemix.net',
    'port': '50001',
    'url': 'https://undefined'
}
schema="CHICAGO"

In [ ]:
dsn = (
    "DRIVER={{IBM DB2 ODBC DRIVER}};"
    "DATABASE={0};"
    "HOSTNAME={1};"
    "PORT={2};"
    "PROTOCOL=TCPIP;"
    "SECURITY=ssl;" 
    "UID={3};"
    "PWD={4};").format(credentials['database'], credentials['host'],
                       credentials['port'], credentials['username'],
                       credentials['password'])

conn = ibm_db.connect(dsn, "", "")
pconn = ibm_db_dbi.Connection(conn)

In [ ]:
# Categorical columns that we want to convert into separate tables
cat_columns = ['TRAFFIC_CONTROL_DEVICE','DEVICE_CONDITION','WEATHER_CONDITION','LIGHTING_CONDITION',
           'FIRST_CRASH_TYPE','TRAFFICWAY_TYPE','ALIGNMENT','ROADWAY_SURFACE_COND','ROAD_DEFECT',
           'REPORT_TYPE','CRASH_TYPE','DAMAGE','PRIM_CONTRIBUTORY_CAUSE','SEC_CONTRIBUTORY_CAUSE',
           'WORK_ZONE_TYPE','MOST_SEVERE_INJURY'
          ]
other_cat_columns = ['POSTED_SPEED_LIMIT','LANE_CNT','NUM_UNITS', 'INJURIES_TOTAL',
                     'CRASH_HOUR','CRASH_DAY_OF_WEEK','CRASH_MONTH']
cat_all = cat_columns + other_cat_columns

## Get the distribution of each categorical attribute
### Build the SQL statement
We need to get all the counts

In [ ]:
# Do the same thing but with Quries to the table.
# I have to build a series of SQL statements and do a UNION ALL on them

query = """
SELECT '{1}' COLNAME, attr.id COLVALUE, COUNT(*) VALCOUNT
FROM {0}.staging_ChicagoAccidents acc, {0}.{1}_table attr
WHERE acc.{1} = attr.description
GROUP BY '{1}', attr.id 
"""

query2 = """
SELECT '{1}' COLNAME, {1} COLVALUE, COUNT(*) VALCOUNT
FROM {0}.staging_ChicagoAccidents acc
GROUP BY '{1}', {1}
"""

sql = ""
for name in cat_columns :
    if (len(sql) > 0 ) :
        sql = sql + "UNION ALL"
    sql = sql + query.format(schema,name)
for name in other_cat_columns :
    sql = sql + "UNION ALL"
    sql = sql + query2.format(schema,name)

stats_pd = pd.read_sql(sql, pconn)
print("Number of records: {0}".format(stats_pd.shape[0]))

### Display the graphs

In [ ]:
nb_rows = math.ceil(len(cat_all) / 2)

fig, axes = plt.subplots(nrows=nb_rows, ncols=2)
fig.set_figheight(75)
fig.set_figwidth(15)
for ix, ax in enumerate(axes.flatten()) :
    if (ix < len(cat_all) ) :
        tmp_pd = stats_pd[stats_pd['COLNAME'] == cat_all[ix]].sort_values(by=['COLVALUE'])
        tmp_pd.plot.bar(ax=ax, x='COLVALUE', y='VALCOUNT',title=cat_all[ix], legend=False)
        ax.set_xlabel('')
    else:
        fig.delaxes(ax)

## Grouping categories
We make the following (arbitrary) decisions:
- TRAFFIC_CONTROL_DEVICE: create 3 groups
- DEVICE_CONDITION: create 3 groups
- WEATHER_CONDITION: create 2 groups
- LIGHTING_CONDITION: create 2 groups
- FIRST_CRASH_TYPE: create 6 groups
- TRAFFICWAY_TYPE: create 2 groups
- ALIGNMENT:ignore this attribute
- ROADWAY_SURFACE_COND: create 2 groups
- ROAD_DEFECT: Create 2 groups
- REPORT_TYPE: keep as-is
- CRASH_TYPE:keep as-is
- DAMAGE: keep as-is
- PRIM_CONTRIBUTORY_CAUSE: create 4 groups
- SEC_CONTRIBUTORY_CAUSE: create 3 groups
- WORK_ZONE_TYPE: ignore this attribute
- MOST_SEVERE_INJURY: ignore this attribute
- POSTED_SPEED_LIMIT: create 2 groups (30 or other)
- LANE_CNT: create 3 groups (2, 4, other)
- NUM_UNITS: ignore this attribute
- INJURIES_TOTAL: ignore this attribute
- CRASH_HOUR: keep as-is
- CRASH_DAY_OF_WEEK: keep as-is
- CRASH_MONTH: keep as-is

## Modify the tables to add a grouping column
For the attributes that already have "side" tables created, we need to modify them to add a column.<br/>
Then we need to populate them according to our decisions above.

In [ ]:
alter_def = """
ALTER TABLE {0}.{1}_TABLE ADD COLUMN grouping INTEGER DEFAULT 0
"""

for col in cat_columns :
    sql = alter_def.format(schema,col)
    cur = pconn.cursor()
    cur.execute(sql)
    print("Table {0}_TABLE altered".format(col))

### Add tables for the other columns
Add the grouping column right away.

In [ ]:
table_def = """
CREATE TABLE {0}.{1}_TABLE (
    id          INT NOT NULL,
    description INT,
    grouping    INT default 0,

    PRIMARY KEY(id)
) ORGANIZE BY ROW;
"""

for col in other_cat_columns :
    sql = table_def.format(schema,col)
    cur = pconn.cursor()
    cur.execute(sql)
    print("Table {0}_TABLE created".format(col))

In [ ]:
# Populate the tables
insert_sql = """
  INSERT INTO {0}.{1}_TABLE(id,description)
  SELECT {1}, {1}
  FROM (
     SELECT distinct {1} 
     FROM {0}.staging_ChicagoAccidents
  )
"""
for col in other_cat_columns :
    sql = insert_sql.format(schema,col)
    cur = pconn.cursor()
    cur.execute(sql)
    print("Table {0}_TABLE populated".format(col))

## Implement our grouping
We created the tables with a default value for grouping. That default value would be for "other".<br/>
This way, we only need to update the exceptions.

In [ ]:
# The two types of SQL statements we need to execute
sql = """
UPDATE {0}.{1}_TABLE
SET grouping = {3}
WHERE id = {2}
"""
sql2 = """
UPDATE {0}.{1}_TABLE
SET grouping = id
"""

In [ ]:
# TRAFFIC_CONTROL_DEVICE: 15 categories. 12-group1, 13-group2, all others group3
d = {'id': [12,13], 'grouping': list(range(1,3))}
cur = pconn.cursor()
for ix in range(len(d['id'])) :
    cur.execute(sql.format(schema,'TRAFFIC_CONTROL_DEVICE',d['id'][ix],d['grouping'][ix]))


In [ ]:
# WEATHER_CONDITION
cur = pconn.cursor()
result = cur.execute(sql.format(schema,'WEATHER_CONDITION',8,1))


In [ ]:
# DEVICE_CONDITION: 8 categories, define 6, 7
d = {'id': [6,7], 'grouping': [1,2]}
cur = pconn.cursor()
for ix in range(len(d['id'])) :
    cur.execute(sql.format(schema,'DEVICE_CONDITION',d['id'][ix],d['grouping'][ix]))

In [ ]:
# LIGHTING_CONDITION
cur = pconn.cursor()
result = cur.execute(sql.format(schema,'LIGHTING_CONDITION',3,1))

In [ ]:
# FIRST_CRASH_TYPE
d = {'id': [4,7,13,14,15], 'grouping': [1,2,3,4,5]}
cur = pconn.cursor()
for ix in range(len(d['id'])) :
    cur.execute(sql.format(schema,'FIRST_CRASH_TYPE',d['id'][ix],d['grouping'][ix]))

In [ ]:
# TRAFFICWAY_TYPE: create 2 groups 
cur = pconn.cursor()
result = cur.execute(sql.format(schema,'TRAFFICWAY_TYPE',9,1))

In [ ]:
# ALIGNMENT:ignore this attribute
# ROADWAY_SURFACE_COND: create 2 groups
cur = pconn.cursor()
result = cur.execute(sql.format(schema,'ROADWAY_SURFACE_COND',2,1))
# ROAD_DEFECT: Create 2 groups
result = cur.execute(sql.format(schema,'ROAD_DEFECT',6,1))


In [ ]:
# REPORT_TYPE: keep as-is
cur = pconn.cursor()
result = cur.execute(sql2.format(schema,'REPORT_TYPE'))
# CRASH_TYPE:keep as-is
result = cur.execute(sql2.format(schema,'CRASH_TYPE'))
# DAMAGE: keep as-is
result = cur.execute(sql2.format(schema,'DAMAGE'))


In [ ]:
# PRIM_CONTRIBUTORY_CAUSE: create 4 groups
d = {'id': [8,11,22], 'grouping': [1,2,3]}
cur = pconn.cursor()
for ix in range(len(d['id'])) :
    cur.execute(sql.format(schema,'PRIM_CONTRIBUTORY_CAUSE',d['id'][ix],d['grouping'][ix]))

In [ ]:
# SEC_CONTRIBUTORY_CAUSE: create 3 groups
d = {'id': [30,38], 'grouping': [1,2]}
cur = pconn.cursor()
for ix in range(len(d['id'])) :
    cur.execute(sql.format(schema,'SEC_CONTRIBUTORY_CAUSE',d['id'][ix],d['grouping'][ix]))

In [ ]:
# WORK_ZONE_TYPE: ignore this attribute
# MOST_SEVERE_INJURY: ignore this attribute
# POSTED_SPEED_LIMIT: create 2 groups (30 or other)
cur = pconn.cursor()
result = cur.execute(sql.format(schema,'POSTED_SPEED_LIMIT',30,1))

In [ ]:
# LANE_CNT: create 3 groups (2, 4, other)
d = {'id': [2,4], 'grouping': [1,2]}
cur = pconn.cursor()
for ix in range(len(d['id'])) :
    cur.execute(sql.format(schema,'LANE_CNT',d['id'][ix],d['grouping'][ix]))

In [ ]:
# NUM_UNITS: ignore this attribute
# INJURIES_TOTAL
# CRASH_HOUR: keep as-is
cur = pconn.cursor()
result = cur.execute(sql2.format(schema,'CRASH_HOUR'))

In [ ]:
# CRASH_DAY_OF_WEEK: keep as-is
cur = pconn.cursor()
result = cur.execute(sql2.format(schema,'CRASH_DAY_OF_WEEK'))

In [ ]:
# CRASH_MONTH: keep as-is
cur = pconn.cursor()
result = cur.execute(sql2.format(schema,'CRASH_MONTH'))

## Correlation
We can also get an idea of how numerical attributes relate to each other.

We could read all the records into a Pandas dataframe and then do the correlation.
Instead, we use a different interface to the db2 table, **`ibmdbpy`**, to get a reference
to the table and have the correlation done in the database server.

This is a lot faster and efficient than reading all the data and then do the correlation.

In [ ]:
from ibmdbpy import IdaDataBase, IdaDataFrame

dsn2 = 'DASHDB;Database={0};Hostname={1};Port={2};PROTOCOL=TCPIP;SECURITY=ssl;UID={3};PWD={4}'.\
format(credentials['database'], credentials['host'],
                       credentials['port'], credentials['username'],
                       credentials['password'])
idadb = IdaDataBase(dsn=dsn2)
ida_df = IdaDataFrame(idadb, '{0}.STAGING_CHICAGOACCIDENTS'.format(schema))
corr_pd = ida_df.corr()
# idadb.close()

In [ ]:
import matplotlib.pyplot as plt
f = plt.figure(figsize=(12, 12))
plt.matshow(corr_pd, fignum=f.number)
plt.xticks(range(corr_pd.shape[1]), corr_pd.columns, fontsize=8, rotation=45)
plt.yticks(range(corr_pd.shape[1]), corr_pd.columns, fontsize=8)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=8)

# Create the final table
First, go get the column definitions so we can use the proper data types.

The column type seems to be followed by a space, we need to accomodate for that. Also, the TIMESTAMP type is listed as TIMESTMP so we need to cover this problem.

In [ ]:
sql = """
SELECT NAME,COLTYPE,LENGTH,SCALE, NULLS
FROM SYSIBM.SYSCOLUMNS
WHERE TBNAME = 'STAGING_CHICAGOACCIDENTS'
AND   TBCREATOR = '{0}'
ORDER BY COLNO;
""".format(schema)
tabdef_pd = pd.read_sql(sql, pconn)
tabdef_pd.head(5)

In [ ]:
# Create the table. This is just helper code where I added the definition of the primary key

column_types = {"VARCHAR": "VARCHAR", "CHAR": "CHAR", "INTEGER":"INTEGER","DOUBLE":"DOUBLE", "TIMESTMP": "TIMESTAMP"}

sql = "CREATE TABLE {0}.ChicagoAccidents (\n".format(schema)

for row in tabdef_pd.iterrows() :
    if row[1]['NAME'] in cat_columns :
        sql = sql + "  {0:33} INTEGER REFERENCES {1}.{2}_TABLE(ID),\n".format(row[1]['NAME'] + "_ID",schema,row[1]['NAME'])
    else :
        # we need to add the type, length, and if not null
        sql = sql + "  {0:33} {1}".format(row[1]['NAME'],column_types[row[1]['COLTYPE'].strip()] )
        if row[1]['COLTYPE'].find('CHAR') > -1 :
            sql = sql + "({0}) ".format(row[1]['LENGTH'])
        if row[1]['NULLS'] == 'N' :
            sql = sql + "NOT NULL "
        sql = sql + ",\n"

sql = sql + "\n  PRIMARY KEY(RD_NO)\n) ORGANIZE BY ROW;"
# print(sql)
cur = pconn.cursor()
cur.execute(sql)

## Populate the new table
We build the SQL statement programmatically

In [ ]:
sql = "INSERT INTO {0}.ChicagoAccidents \nSELECT ".format(schema)

# Identify the columns
for row in tabdef_pd.iterrows() :
    if row[1]['NAME'] in cat_columns :
        sql = sql + "{0}.{1}_table.id as {1}_ID,".format(schema,row[1]['NAME'])
    else :
        sql = sql + "{0},".format(row[1]['NAME'])

sql = sql[:-1] + "\n FROM {0}.staging_ChicagoAccidents,".format(schema)

# Add the other tables
for name in cat_columns :
    sql = sql + "{0}.{1}_table,".format(schema,name)
sql = sql[:-1] + "\n WHERE "

# Add the conditions
for name in cat_columns :
    sql = sql + "{0}.staging_ChicagoAccidents.{1} = {0}.{1}_table.description\nAND ".format(schema,name) 
sql = sql[:-4] + ";"

# print(sql)
cur = pconn.cursor()
cur.execute(sql)